# AP3 - Pattern Recognition
Implementation of a decision tree for classifying a database with categorical attributes.

> Name: Jonas Carvalho Fortes

> Mat: 494513

## Load Dataset

In [457]:
import pandas as pd
from scipy.io import loadmat

dataset = loadmat('data/Dataset.mat')
dataset = pd.DataFrame(dataset['Dataset'])
print(f'Dataset shape: {dataset.shape}')



Dataset shape: (876, 4)


In [458]:
print('Dataset:')
dataset

Dataset:


,0,1,2,3
0,1,0,0,1
1,0,0,0,1
2,0,0,0,0
3,0,0,0,0
4,0,0,0,1
...,...,...,...,...
871,1,1,0,1
872,1,1,0,1
873,1,1,0,1
874,1,0,1,1


## Helper functions and classes

### Auxiliary Functions

In [459]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import numpy as np 

# Menor valor de entropia possível que não seja zero (usado para evitar divisão por zero)
EPSILON = np.finfo('float32').eps

# Calcula a entropia de um conjunto de rótulos
def entropy(labels):
    """
    Calcula a entropia de um conjunto de rótulos.
    
    Args:
        labels (np.ndarray): Array contendo os rótulos dos dados (0 ou 1).
    
    Returns:
        float: Entropia do conjunto de rótulos.
    """
    # Calcula a frequência de cada classe
    values, counts = np.unique(labels, return_counts=True)
    
    # Calcula a probabilidade de cada classe
    probabilities = counts / len(labels)
    
    # Calcula a entropia usando a fórmula da entropia
    entropy_value = -np.sum(probabilities * np.log2(probabilities + EPSILON))
    
    return entropy_value

# Calcula o ganho de informação de uma divisão dos dados
def information_gain(original_labels, left_labels, right_labels):
    """
    Calcula o ganho de informação de uma divisão dos dados.
    
    Args:
        original_labels (np.ndarray): Rótulos antes da divisão.
        left_labels (np.ndarray): Rótulos do subconjunto à esquerda.
        right_labels (np.ndarray): Rótulos do subconjunto à direita.
    
    Returns:
        float: Ganho de informação da divisão.
    """
    # Entropia antes da divisão
    original_entropy = entropy(original_labels)
    
    # Calcula a entropia ponderada após a divisão
    total_len = len(original_labels)
    left_prob = len(left_labels) / total_len
    right_prob = len(right_labels) / total_len
    
    # Entropia ponderada após a divisão
    weighted_entropy = (left_prob * entropy(left_labels)) + (right_prob * entropy(right_labels))
    
    # Ganho de informação
    gain = original_entropy - weighted_entropy
    
    return gain

# Obtém a classe majoritária de um conjunto de rótulos
def get_majority_class(labels):
    """
    Obtém a classe majoritária de um conjunto de rótulos.
    
    Args:
        labels (np.ndarray): Array contendo os rótulos dos dados (0 ou 1).
    
    Returns:
        int: A classe majoritária.
    """
    # Conta a frequência de cada classe
    values, counts = np.unique(labels, return_counts=True)
    
    # Retorna a classe com a maior frequência
    majority_class = values[np.argmax(counts)]
    
    return majority_class

# Função para realizar K-fold cross-validation com a árvore de decisão
def k_fold_cross_validation(X, y, k_folds, decision_tree_class):
    """
    Realiza K-fold cross-validation com a árvore de decisão 
    e retorna as métricas médias de cada fold.
    
    Args: 
        X (np.ndarray): Dados de entrada.
        y (np.ndarray): Rótulos dos dados.
        k_folds (int): Número de folds.
        decision_tree_class (class): Classe da árvore de decisão.
    
    Returns:
        dict: Dicionário contendo as métricas médias de cada fold.
        
    """
    
    # Dividir os dados em K folds
    fold_size = len(y) // k_folds
    folds_X = [X[i * fold_size:(i + 1) * fold_size] for i in range(k_folds)]
    folds_y = [y[i * fold_size:(i + 1) * fold_size] for i in range(k_folds)]

    # Listas para armazenar as métricas de cada fold
    accuracies = []
    sensitivities = []
    specificities = []
    precisions = []
    f1_scores = []

    # Iterar sobre cada fold
    for i in range(k_folds):
        
        # Separar os dados em treino e teste
        X_test = folds_X[i]
        y_test = folds_y[i]
        
        # Concatenar os outros folds para treino
        X_train = np.concatenate([folds_X[j] for j in range(k_folds) if j != i], axis=0)
        y_train = np.concatenate([folds_y[j] for j in range(k_folds) if j != i], axis=0)

        # Criar uma nova instância da árvore de decisão e treinar com os dados de treinamento
        tree = decision_tree_class()
        tree.fit(X_train, y_train) # Treinar a árvore

        # Fazer predições nos dados de teste
        y_pred = [tree.predict(X_test[j]) for j in range(len(X_test))]

        # Calcular a matriz de confusão
        """
        tn: verdadeiros negativos, 
        fp: falsos positivos, 
        fn: falsos negativos, 
        tp: verdadeiros positivos
        """
        tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()

        # Calcular as métricas
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        sensitivity = recall_score(y_test, y_pred)  # Sensibilidade é o mesmo que recall
        specificity = tn / (tn + fp)  # Especificidade: proporção de verdadeiros negativos
        f1 = f1_score(y_test, y_pred)

        # Armazenar as métricas
        accuracies.append(accuracy)
        precisions.append(precision)
        sensitivities.append(sensitivity)
        specificities.append(specificity)
        f1_scores.append(f1)

    # Calcular a média das métricas
    mean_accuracy = float(np.mean(accuracies).round(2))
    mean_precision = float(np.mean(precisions).round(2))
    mean_sensitivity = float(np.mean(sensitivities).round(2))
    mean_specificity = float(np.mean(specificities).round(2))
    mean_f1_score = float(np.mean(f1_scores).round(2))

    return {
        'accuracy': f'{mean_accuracy*100}%',
        'precision': f'{mean_precision*100}%',
        'sensitivity': f'{mean_sensitivity*100}%',
        'specificity': f'{mean_specificity*100}%',
        'f1_score': f'{mean_f1_score*100}%'
    }
   

### Decision Tree Modeling

In [460]:

class Node:
    """
    Nó genérico de uma árvore de decisão.
    """
    
    def __init__(self, left, right, data=None, feature_idx=None, 
                 feature_name=None, criterion_value=None,
                 _result=None, class_name=None):
        
        self.left: Node = left
        self.right: Node = right

        self.data = data
        self.feature_idx: int = feature_idx
        self.feature_name: str = feature_name
        self.criterion_value: float = criterion_value
        self.n_sample: int = len(data) if data is not None else 0
        self._result = _result
        self.class_name: str = class_name

    def predict(self, x):
        if x[self.feature_idx] == 0:
            return self.left.predict(x)
        else:  # Aqui x[self.feature_idx] deve ser 1
            return self.right.predict(x)


class LeafNode(Node):
    
    """
    Nó folha de uma árvore de decisão. 
    Este nó não possui filhos e é usado para parar a recursão.
    É uma extensão da classe Node.
    """
    
    def __init__(self, data, criterion_value, _result, class_name):
        super().__init__(None, None, data=data, 
                         criterion_value=criterion_value, 
                         _result=_result, class_name=class_name)

    def predict(self, X=None):
        return self._result
    
    
class DecisionTree: 
    """
    Implementação de uma árvore de decisão binária.
    Usando a entropia como critério de divisão.
    """
    
    def __init__(self, feature_names=['Empregado', 'Devedor', 'Salário>5SM'], class_names=['não', 'sim']):
        self.root_node: Node = Node(None, None)
        self.feature_names = feature_names
        self.class_names = class_names
    
    def fit(self, X: np.ndarray, y: np.ndarray):
         # Cria um array de dados que inclui atributos e rótulos
        data = np.hstack((X, y.reshape(-1, 1)))
        # Chama o método grow para construir toda a árvore a partir da raiz
        self.root_node = self._grow(data)
    
    def predict(self, X):
        return self.root_node.predict(X)

    def _split_data(self, feature_data, labels):
        """
        Divide os dados com base nos valores categóricos (0 ou 1) do atributo.
        """
        # Dividindo à esquerda (valor 0)
        left_indices = feature_data == 0
        left_labels = labels[left_indices]

        # Dividindo à direita (valor 1)
        right_indices = feature_data == 1
        right_labels = labels[right_indices]

        return (left_labels, right_labels)
    
    def _best_feature(self, data, feature_idxs):
        """
        Identifica o melhor atributo binário para dividir os dados.
        """
        # Inicializa as variáveis de controle
        max_gain = -np.inf
        selected_feature = None

        # Itera sobre os índices dos atributos
        for feature_idx in feature_idxs:
            # Obtém os dados do atributo
            feature_data = data[:, feature_idx]
            labels = data[:, -1]
            
            # Avalia o ganho de informação para o atributo atual
            left_labels, right_labels = self._split_data(feature_data, labels)
            
            # Calcula o ganho de informação
            gain = information_gain(labels, left_labels, right_labels)

            # Atualiza o atributo selecionado se o ganho for maior
            if gain > max_gain:
                max_gain = gain
                selected_feature = feature_idx
                
        # Retorna o índice do atributo selecionado e o ganho de informação
        return selected_feature, max_gain
    
    def _grow(self, data, used_features=None):
        """
        Constrói recursivamente a árvore de decisão.
        """
        if used_features is None:
            used_features = set()

        # Define o uso das funções de entropia e classe majoritária
        compute_criterion_value = entropy
        get_result = get_majority_class

        # Obtém os rótulos dos dados
        y = data[:, -1]
        
        # Calcula a entropia dos dados
        criterion_value = compute_criterion_value(y)
        
        # Obtém a classe majoritária dos dados
        class_result = get_result(y)
        
        # Define o nome da classe (usado para visualização)
        class_name = self.class_names[class_result] if self.class_names else f"{class_result:.4f}"

        # Critério de parada: parar se a entropia for zero (dados puros) ou se todos os atributos já tiverem sido usados
        if criterion_value < EPSILON or len(used_features) >= 3:
            # Retorna um nó folha (para parar a recursão)
            return LeafNode(data, criterion_value=criterion_value, 
                            _result=class_result, class_name=class_name)

        # Seleciona o melhor atributo que não foi usado ainda para dividir os dados
        feature_idxs = [i for i in np.arange(data.shape[-1] - 1) if i not in used_features]
        selected_feature, _ = self._best_feature(data, feature_idxs)

        # Critério de parada: parar se não houver mais atributos para dividir
        if selected_feature is None:
            return LeafNode(data, criterion_value=criterion_value, 
                            _result=class_result, class_name=class_name)

        # Adiciona o atributo selecionado ao conjunto de atributos usados 
        # (para evitar a repetição do mesmo atributo)
        used_features.add(selected_feature)

        # Divide os dados com base no atributo selecionado
        left_data = data[data[:, selected_feature] == 0]
        right_data = data[data[:, selected_feature] == 1]

        # Cria os nós filhos recursivamente
        left_node = self._grow(left_data, used_features=used_features)
        right_node = self._grow(right_data, used_features=used_features)
        
        # Retorna um nó de decisão com os nós filhos criados
        return Node(left_node, 
                    right_node,
                    data, 
                    selected_feature, 
                    feature_name=self.feature_names[selected_feature] if any(self.feature_names) else "Name not defined",
                    criterion_value=criterion_value,
                    _result=class_result, class_name=class_name)

## Evaluation Metrics with K-folds


In [461]:
# Separando atributos e rótulos
X = dataset.iloc[:, :-1].values  # Atributos (todas as colunas exceto a última)
y = dataset.iloc[:, -1].values   # Rótulo (última coluna)

# Nomes dos atributos e classes
feature_names = ["Empregado", "Devedor", "Salário acima de 5SM"]
class_names = ["não", "sim"]

# Criando e treinando a árvore de decisão
tree = DecisionTree(feature_names=feature_names, class_names=class_names)
tree.fit(X, y)

# Fazendo a validação cruzada com a árvore de decisão usando 10 folds
metrics = k_fold_cross_validation(X, y, k_folds=10, decision_tree_class=DecisionTree)

#Imprimindo as métricas
print(f'Métricas (Média de 10 folds):\n{metrics}')



Métricas (Média de 10 folds):
{'accuracy': '89.0%', 'precision': '84.0%', 'sensitivity': '94.0%', 'specificity': '84.0%', 'f1_score': '89.0%'}
